In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import pandas as pd
from typing import Any
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D
from keras.optimizers import SGD
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
def one_hot_encode(sequence: str,
                   alphabet: str = 'ACGT',
                   neutral_alphabet: str = 'N',
                   neutral_value: Any = 0,
                   dtype=np.float64) -> np.ndarray:
  """One-hot encode sequence."""
  def to_uint8(string):
    return np.frombuffer(string.encode('ascii'), dtype=np.uint8)
  hash_table = np.zeros((np.iinfo(np.uint8).max, len(alphabet)), dtype=dtype)
  hash_table[to_uint8(alphabet)] = np.eye(len(alphabet), dtype=dtype)
  hash_table[to_uint8(neutral_alphabet)] = neutral_value
  hash_table = hash_table.astype(dtype)
  return hash_table[to_uint8(sequence)]

In [ ]:
filename = '/home/xlxiang/biostatistics/data/Rregression/x-2k_sequence.fa'
f = open(filename)
lines = f.readlines()
f.close()
seqs = [lines[i].strip() for i in range(len(lines)) if i  % 2 != 0]
seq = [one_hot_encode(i) for i in seqs]

In [ ]:
y = pd.read_csv('/home/xlxiang/biostatistics/data/Rregression/y.csv',index_col=0)
y = np.log2(y['TPM'].values+1)

In [ ]:
#计算训练集和测试集的pearsonr和r2
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
def calculate_result(model,x_train, y_train,x_test, y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    train_pearsonr = pearsonr(y_train, y_pred_train)[0]
    test_pearsonr = pearsonr(y_test, y_pred_test)[0]
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    print("train_pearsonr: {0:.3f}, test_pearsonr: {1:.3f}".format(train_pearsonr,test_pearsonr))
    print("train_r2: {0:.3f}, test_r2: {1:.3f}".format(train_r2, test_r2))

    return None

In [ ]:
def Model(a,b):
    model = Sequential()
    model.add(
        Conv1D(filters=int(b),
               kernel_size=int(a),
               padding='same',
               activation='relu',
               input_shape=(2000, 4)))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    return model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(seq),
                                                    y,
                                                    train_size=0.8,
                                                    random_state=2**10 * 6)
x_train, x_valid, y_train, y_valid = train_test_split(x_train,
                                                    y_train,
                                                    train_size=0.9,
                                                    random_state=2**10 * 6)
print(len(x_train))
print(len(x_valid))
print(len(x_test))

In [ ]:
batch_size = 64
dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
dataset_valid = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(batch_size)
dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [9]:
best_pear = 0
for filter_len in range(7,21,2):
    for filter_num in [8,16,32,64]:
        model = Model(filter_len,filter_num)
        sgd = SGD(lr=0.001, decay=1e-5, momentum=0.9, nesterov=True)
        model.compile(loss='mse',
                      optimizer=sgd)
        checkpoint = ModelCheckpoint(filepath= '/home/xlxiang/biostatistics/code/regression/deepbind.h5',
                                     save_best_only=False,
                                     monitor='val_loss',mode='auto')
        early_stopping =EarlyStopping(monitor='val_loss', patience=5)
        model.fit(dataset_train,
                  epochs=100,
                  validation_data=dataset_valid,
                callbacks=[early_stopping,checkpoint])
        print('model fit over!')
        yt_pred = []
        y_test_all = []
        for i in dataset_test:
            y_test_all.extend(i[1].numpy().ravel().tolist())
            yt_pred.extend(model.predict(i[0]).ravel().tolist())
        pear = pearsonr(yt_pred,y_test_all)[0]
        if pear>best_pear:
              best_pear=pear
              best_filter_len = filter_len
              best_filter_num = filter_num
print("the best filter_len: {}, the best filter_num: {}.".format(best_filter_len,best_filter_num))

2023-06-01 21:13:09.626509: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2568192000 exceeds 10% of free system memory.
2023-06-01 21:13:09.904717: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2568192000 exceeds 10% of free system memory.


Epoch 1/100


2023-06-01 21:13:10.455534: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-06-01 21:13:10.458025: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3696000000 Hz
2023-06-01 21:13:10.522657: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-06-01 21:13:10.716081: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2023-06-01 21:13:11.547134: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2023-06-01 21:13:11.700742: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


627/627 [==============================] - 9s 9ms/step - loss: 6.6811 - val_loss: 4.1846


2023-06-01 21:13:19.562590: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2568192000 exceeds 10% of free system memory.


Epoch 2/100
627/627 [==============================] - 5s 7ms/step - loss: 4.3038 - val_loss: 4.0513


2023-06-01 21:13:24.385033: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2568192000 exceeds 10% of free system memory.


Epoch 3/100
627/627 [==============================] - 5s 7ms/step - loss: 3.9457 - val_loss: 4.0356
Epoch 4/100
627/627 [==============================] - 5s 8ms/step - loss: 3.7154 - val_loss: 4.0505
Epoch 5/100
627/627 [==============================] - 5s 7ms/step - loss: 3.4005 - val_loss: 4.1315
Epoch 6/100
627/627 [==============================] - 5s 7ms/step - loss: 3.0581 - val_loss: 4.2583
Epoch 7/100
627/627 [==============================] - 5s 7ms/step - loss: 2.6813 - val_loss: 4.4236
Epoch 8/100
627/627 [==============================] - 5s 7ms/step - loss: 2.2230 - val_loss: 4.5450
model fit over!
Epoch 1/100
627/627 [==============================] - 9s 10ms/step - loss: 6.7034 - val_loss: 4.1608
Epoch 2/100
627/627 [==============================] - 5s 8ms/step - loss: 4.2113 - val_loss: 4.0302
Epoch 3/100
627/627 [==============================] - 5s 9ms/step - loss: 3.7717 - val_loss: 4.0557
Epoch 4/100
627/627 [==============================] - 5s 9ms/step - loss:

627/627 [==============================] - 10s 12ms/step - loss: 6.7954 - val_loss: 4.1886
Epoch 2/100
627/627 [==============================] - 6s 10ms/step - loss: 4.1180 - val_loss: 4.0652
Epoch 3/100
627/627 [==============================] - 6s 10ms/step - loss: 3.4229 - val_loss: 4.2019
Epoch 4/100
627/627 [==============================] - 6s 10ms/step - loss: 2.6612 - val_loss: 4.4491
Epoch 5/100
627/627 [==============================] - 6s 10ms/step - loss: 2.0459 - val_loss: 4.6350
Epoch 6/100
627/627 [==============================] - 6s 10ms/step - loss: 1.6172 - val_loss: 4.4805
Epoch 7/100
627/627 [==============================] - 6s 10ms/step - loss: 1.3438 - val_loss: 4.4889
model fit over!
Epoch 1/100
627/627 [==============================] - 9s 10ms/step - loss: 7.0228 - val_loss: 4.1855
Epoch 2/100
627/627 [==============================] - 5s 8ms/step - loss: 4.2858 - val_loss: 4.0404
Epoch 3/100
627/627 [==============================] - 5s 8ms/step - loss: 3.8

627/627 [==============================] - 6s 10ms/step - loss: 2.2686 - val_loss: 4.4595
Epoch 7/100
627/627 [==============================] - 6s 10ms/step - loss: 1.7735 - val_loss: 4.5213
model fit over!
Epoch 1/100
627/627 [==============================] - 9s 11ms/step - loss: 6.8356 - val_loss: 4.1601
Epoch 2/100
627/627 [==============================] - 5s 9ms/step - loss: 4.1425 - val_loss: 4.0073
Epoch 3/100
627/627 [==============================] - 5s 9ms/step - loss: 3.5059 - val_loss: 4.0756
Epoch 4/100
627/627 [==============================] - 5s 8ms/step - loss: 2.9860 - val_loss: 4.2257
Epoch 5/100
627/627 [==============================] - 5s 8ms/step - loss: 2.3994 - val_loss: 4.4927
Epoch 6/100
627/627 [==============================] - 5s 8ms/step - loss: 1.8923 - val_loss: 4.5385
Epoch 7/100
627/627 [==============================] - 5s 8ms/step - loss: 1.4966 - val_loss: 4.5038
model fit over!
Epoch 1/100
627/627 [==============================] - 10s 13ms/step

## the best filter_len: 11, the best filter_num: 8

In [10]:
model = Model(11,8)
sgd = SGD(lr=0.001, decay=1e-5, momentum=0.9, nesterov=True)
model.compile(loss='mse',
              optimizer=sgd)
checkpoint = ModelCheckpoint(filepath= '/home/xlxiang/biostatistics/code/regression/deepbind.h5',
                             save_best_only=False,
                             monitor='val_loss',mode='auto')
early_stopping =EarlyStopping(monitor='val_loss', patience=5)
model.fit(dataset_train,
          epochs=100,
          validation_data=dataset_valid,
        callbacks=[early_stopping,checkpoint])
print('model fit over!')

Epoch 1/100
627/627 [==============================] - 5s 7ms/step - loss: 7.1131 - val_loss: 4.4025
Epoch 2/100
627/627 [==============================] - 5s 7ms/step - loss: 4.3600 - val_loss: 4.1442
Epoch 3/100
627/627 [==============================] - 5s 7ms/step - loss: 3.9599 - val_loss: 4.0589
Epoch 4/100
627/627 [==============================] - 5s 7ms/step - loss: 3.7114 - val_loss: 4.0400
Epoch 5/100
627/627 [==============================] - 5s 8ms/step - loss: 3.3976 - val_loss: 4.1340
Epoch 6/100
627/627 [==============================] - 5s 8ms/step - loss: 3.0088 - val_loss: 4.2635
Epoch 7/100
627/627 [==============================] - 5s 8ms/step - loss: 2.6611 - val_loss: 4.4011
Epoch 8/100
627/627 [==============================] - 5s 8ms/step - loss: 2.2069 - val_loss: 4.5117
Epoch 9/100
627/627 [==============================] - 5s 8ms/step - loss: 1.8752 - val_loss: 4.7277
model fit over!


In [28]:
#计算训练集和测试集的pearsonr和r2
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
def calculate_result(model,x_train, y_train,x_test, y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    train_pearsonr = pearsonr(y_train, y_pred_train)[0]
    test_pearsonr = pearsonr(y_test, y_pred_test)[0]
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    print("train_pearsonr: {0:.3f}, test_pearsonr: {1:.3f}".format(train_pearsonr[0],test_pearsonr[0]))
    print("train_r2: {0:.3f}, test_r2: {1:.3f}".format(train_r2, test_r2))

    return None

In [29]:
calculate_result(model,x_valid, y_valid,x_test, y_test)

train_pearsonr: 0.696, test_pearsonr: 0.704
train_r2: 0.459, test_r2: 0.478
